In [1]:
import sys

#!{sys.executable} -m pip install -U --pre "mxnet>=1.7.0b20200713, <2.0.0" -f https://sxjscience.github.io/KDD2020/


In [2]:
#!{sys.executable} -m pip install autogluon

In [3]:
#!{sys.executable} -m pip install bokeh==2.0.1

In [4]:
import autogluon as ag
from autogluon import TabularPrediction as task

In [5]:
###READ IN DATA
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style('darkgrid')
plt.style.use('seaborn-poster')

/home/russell/anaconda3/envs/autoglu/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
data = pd.read_csv("/home/russell/Documents/GitHub/InterviewPrep/creditCards/cc_info.csv")


/home/russell/anaconda3/envs/autoglu/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
#!{sys.executable} -m pip install pandas-profiling[notebook]  #only run once to download/install

In [8]:
from pandas_profiling import ProfileReport
profile = ProfileReport(data, title='Pandas Profiling Report', explorative=True)


In [9]:
profile.to_widgets()

/home/russell/anaconda3/envs/autoglu/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
data.shape

/home/russell/anaconda3/envs/autoglu/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(8950, 18)

In [11]:
label_column = 'MINIMUM_PAYMENTS'

In [12]:
nomissings=data.dropna(subset=['MINIMUM_PAYMENTS']) #drop rows with missing MINIMUM_PAYMENTS values
train = nomissings.sample(frac=0.75, random_state=99)
# you can't simply split 0.75 and 0.25 without overlapping
# this code tries to find that train = 75% and test = 25%
test = nomissings.loc[~nomissings.index.isin(train.index), :]

In [13]:
predictor = task.fit(train_data=train, tuning_data=test,
                     label=label_column,eval_metric = "r2")


No output_directory specified. Models will be saved in: AutogluonModels/ag-20201007_185442/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20201007_185442/
AutoGluon Version:  0.0.13
Train Data Rows:    6478
Train Data Columns: 18
Tuning Data Rows:    2159
Tuning Data Columns: 18
Preprocessing data ...
Here are the first 10 unique label values in your data:  [70.384975, 176.610605, 513.7639320000001, 874.422722, 1220.5291710000001, 165.041701, 271.364415, 561.912557, 153.53184199999998, 601.214943]
AutoGluon infers your prediction problem is: regression  (because dtype of label-column == float and many unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

Feature Generator processed 8637 data points with 16 features
Original Features (raw dtypes):
	object features: 1
	float64 features: 13
	int64 features: 3
Original Feat

In [15]:
performance = predictor.evaluate(test)


Predictive performance on given dataset: r2 = 0.4974798672023729


In [16]:
results = predictor.fit_summary()

/home/russell/anaconda3/envs/autoglu/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   weighted_ensemble_k0_l1   0.497480       0.077351  44.534966                0.001081           0.464597            1       True          9
1        NeuralNetRegressor   0.496933       0.059563  42.400362                0.059563          42.400362            0       True          7
2         LightGBMRegressor   0.312182       0.016708   1.670007                0.016708           1.670007            0       True          5
3         CatboostRegressor   0.296999       0.007457   4.279346                0.007457           4.279346            0       True          6
4   LightGBMRegressorCustom   0.292160       0.020781   1.811275                0.020781           1.811275            0       True          8
5  RandomForestRegressorMSE   0.268681       0.082410   5.898917                

In [17]:
missers = data.loc[~data.index.isin(nomissings.index), :] 
#from the *data* df, pull those rows that 
#weren't included once we dropped rows that were missing MIN Payment values
predicted_MINIMUM_PAYMENTS = predictor.predict(missers)

/home/russell/anaconda3/envs/autoglu/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
missers['MINIMUM_PAYMENTS']=predicted_MINIMUM_PAYMENTS #plug-in interpreted values, obtained from autogluon

combined=nomissings.append(missers)
#recombine missers with nonmissers, creating a more robust dataset to model downstream analyses

/home/russell/anaconda3/envs/autoglu/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-18-a4e8d88483aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missers['MINIMUM_PAYMENTS']=predicted_MINIMUM_PAYMENTS #plug-in interpreted values, obtained from autogluon


In [19]:
combined

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.40,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.00,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.00,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.00,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12
5,C10006,1809.828751,1.000000,1333.28,0.00,1333.28,0.000000,0.666667,0.000000,0.583333,0.000000,0,8,1800.0,1400.057770,2407.246035,0.000000,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8919,C19160,14.524779,0.333333,152.00,152.00,0.00,0.000000,0.333333,0.333333,0.000000,0.000000,0,2,1500.0,0.000000,35.184391,0.000000,6
8929,C19170,371.527312,0.333333,0.00,0.00,0.00,1465.407927,0.000000,0.000000,0.000000,0.166667,5,0,1500.0,0.000000,132.262924,0.000000,6
8935,C19176,183.817004,1.000000,465.90,0.00,465.90,0.000000,1.000000,0.000000,0.833333,0.000000,0,6,1500.0,0.000000,350.997253,0.000000,6
8944,C19185,193.571722,0.833333,1012.73,1012.73,0.00,0.000000,0.333333,0.333333,0.000000,0.000000,0,2,4000.0,0.000000,70.037422,0.000000,6


In [20]:
c_profile = ProfileReport(combined, title='Pandas Profiling Report', explorative=True)
c_profile.to_widgets()